## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [2]:
using StatsBase
using Distributions
using Plots

In [3]:
using TensorFlow
using MLDataUtils

In [4]:
using DensityEstimationML

In [5]:
immutable NeuralDensityEstimator
    sess::Session
    
    #Network nodes
    optimizer::Tensor
    conditioner::Tensor
    t::Tensor
    pdf::Tensor
end

In [6]:
function Distributions.pdf(est::NeuralDensityEstimator, t::Real)
    ts = reshape([t], (1,1))
    pdf(est, ts) |> first
end

function Distributions.pdf(est::NeuralDensityEstimator, ts::AbstractVector)
    gr = est.sess.graph
    run(est.sess, est.pdf, Dict(est.t=>ts')) |> vec
end

function Distributions.loglikelihood(est::NeuralDensityEstimator, ts::AbstractVector)
    gr = est.sess.graph
    run(est.sess, gr["loglikelihood"], Dict(est.t=>ts')) |> vec
end


In [7]:
function StatsBase.fit!(estimator::NeuralDensityEstimator, observations;
    epochs = 20)
    
    gr = estimator.sess.graph
    for ii in 1:epochs
        loss_o = run(estimator.sess, 
            [gr["ysmin"],gr["ysmax"],gr["loglikelihood"], gr["working_loss"],
                estimator.optimizer],
            Dict(estimator.t=>observations'))
        ii % 100 == 1 && println("Epoch $ii: loss: $(loss_o)")
    end
    estimator
end

In [8]:
function NeuralDensityEstimator(prob_layer_sizes, support)
    sess = Session(Graph())
    @tf begin
        t = placeholder(Float32, shape=[1, -1])
        smin = constant(reshape([minimum(support)],(1,1)))
        smax = constant(reshape([maximum(support)],(1,1)))
        
        layer_sizes= [1; prob_layer_sizes; 1]
        
        network_fun_stack = Function[Base.identity]       
       
        for ii in 2:length(layer_sizes)
            below_size = layer_sizes[ii-1]
            above_size = layer_sizes[ii]
                       
            Wii = get_variable("W_$ii", [above_size, below_size], Float32)
            Wii2  = Ops.mul(Wii, Wii; name = "W_$(ii)_squared")
            act_fun = if ii!=length(layer_sizes)
                bii = get_variable("b_$ii", [above_size, 1], Float32)
                z -> nn.sigmoid(Wii2*z .+ bii)
            else
                z-> exp(Wii2*z)
            end
            push!(network_fun_stack, z->act_fun(network_fun_stack[ii-1](z)))
        end
        
        network = network_fun_stack[end]

        
        ysmin = TensorFlow.identity(network(smin))
        ysmax = TensorFlow.identity(network(smax))
        yt = network(t)
        
        denominator = (ysmax-ysmin) #area
        numerator = gradients(yt,t)
        pdf =numerator/denominator
        
        
        n_points = TensorFlow.shape(t)[2]
        loglikelihood = reduce_sum(log(numerator)) - n_points.*log(denominator)
        
        area_loss = (1f0.-denominator)^2
        working_loss = -1*loglikelihood + 0.1*area_loss
        
        optimizer = train.minimize(train.AdamOptimizer(), working_loss)
        
        
        # Conditioning
        # Make sure that ysmin~=1, and ysmax~=2
        condition_loss = (1f0 - ysmin)^2 + (2f0 - ysmax)^2
        condition_optimiser = train.minimize(train.AdamOptimizer(;name="adam_cond"), condition_loss)
    end
    
    run(sess, global_variables_initializer())
    
    NeuralDensityEstimator(sess, optimizer, condition_optimiser, t, pdf)
end

NeuralDensityEstimator

In [9]:
"""
    condition(est::NeuralDensityEstimator tol = 1e-15, max_epochs=2_000)
    
"Conditions" the neural density estimate so the support extrema are mapped to 1. and 2.
This improves training by adjusting the area the network has the learn over

"""
function condition!(est::NeuralDensityEstimator, tol = 1e-15, max_epochs=2_000)
    gr = est.sess.graph
    for ii in 1:2_000
        _, ysmin, ysmax, condition_loss = run(est.sess, [est.conditioner, gr["ysmin"],gr["ysmax"], gr["condition_loss"]])
        ii % 50 == 1 && @show (ii, ysmin, ysmax, condition_loss)
        if condition_loss[1] < 1e-15
            break
        end
    end
end


condition!

In [10]:
function demonstration_plot(est, dataset, data=rand(dataset), args...; kwargs...)
    X = minimum(approximate_support(dataset)) : 0.01 : maximum(approximate_support(dataset))
    println("True loglikelihood      = $(loglikelihood(dataset, data))")
    println("Estimated loglikelihood = $(loglikelihood(est, data))")
    estimated_loglikelihood = loglikelihood(dataset, data)
    plot([X], [pdf(est,X), data],
        #xlims= approximate_support(dataset),
        xlims= (first(X), last(X)),
        seriestype = [:path :histogram],
        layout=(2,1),
        legend=false,
        nbins=[1  length(data)÷10],
        args...; kwargs...
    )
end

demonstration_plot (generic function with 2 methods)

In [11]:
function demo(dataset, layers, epochs=20_000)
    data = original_sample(dataset)
    @show loglikelihood(dataset, data)
    est = NeuralDensityEstimator(layers, approximate_support(dataset))
    condition!(est)
    println("Conditioning Done")
    fit!(est, data; epochs=epochs)
    println("Fitting Done")
    demonstration_plot(est, dataset, data)
end

demo (generic function with 2 methods)

In [11]:
demo(GenerateDatasets.Likas1(), [64,64], 20_000)

loglikelihood(dataset, data) = -10450.880952869178


2017-09-13 12:40:36.565336: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-13 12:40:36.565366: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-13 12:40:36.565371: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-09-13 12:40:36.769446: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:893] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2017-09-13 12:40:36.769950: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 0 with propert

(ii, ysmin, ysmax, condition_loss) = (1, [1.00289], [1.00289], [0.994241])
(ii, ysmin, ysmax, condition_loss) = (51, [1.16938], [1.16983], [0.71788])
(ii, ysmin, ysmax, condition_loss) = (101, [1.51805], [1.54786], [0.472808])
(ii, ysmin, ysmax, condition_loss) = (151, [1.49807], [1.61722], [0.394601])
(ii, ysmin, ysmax, condition_loss) = (201, [1.46288], [1.72085], [0.292186])
(ii, ysmin, ysmax, condition_loss) = (251, [1.42253], [1.79787], [0.219392])
(ii, ysmin, ysmax, condition_loss) = (301, [1.38935], [1.84115], [0.17683])
(ii, ysmin, ysmax, condition_loss) = (351, [1.36481], [1.86678], [0.150835])
(ii, ysmin, ysmax, condition_loss) = (401, [1.34586], [1.88389], [0.133103])
(ii, ysmin, ysmax, condition_loss) = (451, [1.33035], [1.89649], [0.119848])
(ii, ysmin, ysmax, condition_loss) = (501, [1.31704], [1.9064], [0.109276])
(ii, ysmin, ysmax, condition_loss) = (551, [1.30521], [1.91459], [0.100451])
(ii, ysmin, ysmax, condition_loss) = (601, [1.29443], [1.92159], [0.0928369])
(ii,

Epoch 6701: loss: Any[[1.84516], [6.97477], [-10909.0], [10910.7], nothing]
Epoch 6801: loss: Any[[1.83588], [6.73233], [-10886.2], [10887.8], nothing]
Epoch 6901: loss: Any[[1.84761], [6.43851], [-10871.6], [10872.9], nothing]
Epoch 7001: loss: Any[[1.877], [6.18669], [-10861.9], [10863.0], nothing]
Epoch 7101: loss: Any[[1.91711], [5.96897], [-10855.3], [10856.2], nothing]
Epoch 7201: loss: Any[[1.96371], [5.7776], [-10850.5], [10851.3], nothing]
Epoch 7301: loss: Any[[2.01434], [5.61008], [-10847.0], [10847.7], nothing]
Epoch 7401: loss: Any[[2.06741], [5.46418], [-10844.3], [10844.9], nothing]
Epoch 7501: loss: Any[[2.12202], [5.33939], [-10842.1], [10842.6], nothing]
Epoch 7601: loss: Any[[2.17753], [5.2347], [-10840.3], [10840.8], nothing]
Epoch 7701: loss: Any[[2.23348], [5.14847], [-10838.8], [10839.2], nothing]
Epoch 7801: loss: Any[[2.28961], [5.07986], [-10837.4], [10837.7], nothing]
Epoch 7901: loss: Any[[2.34564], [5.02632], [-10836.1], [10836.4], nothing]
Epoch 8001: loss

Epoch 17401: loss: Any[[3.42043], [5.04322], [-10729.9], [10730.0], nothing]
Epoch 17501: loss: Any[[3.43516], [5.05401], [-10729.7], [10729.7], nothing]
Epoch 17601: loss: Any[[3.44978], [5.06451], [-10729.4], [10729.4], nothing]
Epoch 17701: loss: Any[[3.46515], [5.07634], [-10729.1], [10729.2], nothing]
Epoch 17801: loss: Any[[3.48115], [5.08923], [-10728.9], [10728.9], nothing]
Epoch 17901: loss: Any[[3.49641], [5.10104], [-10728.6], [10728.7], nothing]
Epoch 18001: loss: Any[[3.51132], [5.11221], [-10728.4], [10728.4], nothing]
Epoch 18101: loss: Any[[3.52686], [5.12472], [-10728.2], [10728.2], nothing]
Epoch 18201: loss: Any[[3.54283], [5.13783], [-10727.9], [10728.0], nothing]
Epoch 18301: loss: Any[[3.5587], [5.15127], [-10727.7], [10727.7], nothing]
Epoch 18401: loss: Any[[3.57241], [5.16103], [-10727.5], [10727.5], nothing]
Epoch 18501: loss: Any[[3.58799], [5.17361], [-10727.3], [10727.3], nothing]
Epoch 18601: loss: Any[[3.60304], [5.18529], [-10727.1], [10727.1], nothing]


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.8117,175.813 39.0374,175.813 39.263,175.813 39.4886,175.813 39.7142,175.813 39.9399,175.813 40.1655,175.813 40.3911,175.813 40.6167,175.813 40.8424,175.813 
 41.068,175.813 41.2936,175.813 41.5193,175.813 41.7449,175.813 41.9705,175.813 42.1961,175.813 42.4218,175.813 42.6474,175.813 42.873,175.813 43.0986,175.813 
 43.3243,175.813 43.5499,175.813 43.7755,175.813 44.0011,175.813 44.2268,175.813 44.4524,175.813 44.678,175.813 44.9036,175.813 45.1293,175.813 45.3549,175.813 
 45.5805,175.813 45.8062,175.813 46.0318,175.813 46.2574,175.813 46.483,175.813 46.7087,175.813 46.9343,175.813 47.1599,175.813 47.3855,175.813 47.6112,175.813 
 47.8368,175.813 48.0624,175.813 48.288,175.813 48.5137,175.813 48.7393,175.813 48.9649,175.813 49.1905,175.813 49.4162,175.813 49.6418,175.813 49.8674,175.813 
 50.0931,175.813 50.3187,175.813 50.5443,175.813 50.7699,175.813 50.9956,175.813 51.2212,175.813 51.4468,175.813 51.6724,175.813 51.8981,175.813 52.1237,175.813 
 52.3493,175.813 52.5749,175.813 52.8006,175.813 53.0262,175.813 53.2518,175.813 53.4774,175.813 53.7031,175.813 53.9287,175.813 54.1543,175.813 54.38,175.813 
 54.6056,175.813 54.8312,175.813 55.0568,175.813 55.2825,175.813 55.5081,175.813 55.7337,175.813 55.9593,175.813 56.185,175.813 56.4106,175.813 56.6362,175.813 
 56.8618,175.813 57.0875,175.813 57.3131,175.813 57.5387,175.813 57.7643,175.813 57.99,175.813 58.2156,175.813 58.4412,175.813 58.6669,175.813 58.8925,175.813 
 59.1181,175.813 59.3437,175.813 59.5694,175.813 59.795,175.813 60.0206,175.813 60.2462,175.813 60.4719,175.813 60.6975,175.813 60.9231,175.813 61.1487,175.813 
 61.3744,175.813 61.6,175.813 61.8256,175.813 62.0512,175.813 62.2769,175.813 62.5025,175.813 62.7281,175.813 62.9538,175.813 63.1794,175.813 63.405,175.813 
 63.6306,175.813 63.8563,175.813 64.0819,175.813 64.3075,175.813 64.5331,175.813 64.7588,175.813 64.9844,175.813 65.21,175.813 65.4356,175.813 65.6613,175.812 
 65.8869,175.812 66.1125,175.812 66.3381,175.812 66.5638,175.812 66.7894,175.812 67.015,175.812 67.2407,175.812 67.4663,175.812 67.6919,175.812 67.9175,175.812 
 68.1432,175.812 68.3688,175.812 68.5944,175.812 68.82,175.812 69.0457,175.812 69.2713,175.812 69.4969,175.812 69.7225,175.812 69.9482,175.812 70.1738,175.812 
 70.3994,175.812 70.625,175.812 70.8507,175.812 71.0763,175.812 71.3019,175.812 71.5276,175.812 71.7532,175.812 71.9788,175.812 72.2044,175.812 72.4301,175.812 
 72.6557,175.812 72.8813,175.812 73.1069,175.812 73.3326,175.812 73.5582,175.812 73.7838,175.812 74.0094,175.811 74.2351,175.811 74.4607,175.811 74.6863,175.811 
 74.912,175.811 75.1376,175.811 75.3632,175.811 75.5888,175.811 75.8145,175.811 76.0401,175.811 76.2657,175.811 76.4913,175.811 76.717,175.811 76.9426,175.811 
 77.1682,175.811 77.3938,175.811 77.6195,175.811 77.8451,175.811 78.0707,175.811 78.2963,175.811 78.522,175.81 78.7476,175.81 78.9732,175.81 79.1989,175.81 
 79.4245,175.81 79.6501,175.81 79.8757,175.81 80.1014,175.81 80.327,175.81 80.5526,175.81 80.7782,175.81 81.0039,175.81 81.2295,175.81 81.4551,175.809 
 81.6807,175.809 81.9064,175.809 82.132,175.809 82.3576,175.809 82.5832,175.809 82.8089,175.809 83.0345,175.809 83.2601,175.809 83.4858,175.809 83.7114,175.808 
 83.937,175.808 84.1626,175.808 84.3883,175.808 84.6139,175.808 84.8395,175.808 85.0651,175.808 85.2908,175.808 85.5164,175.807 85.742,175.807 85.9676,175.807 
 86.1933,175.807 86.4189,175.807 86.6445,175.807 86.8701,175.806 87.0958,175.806 87.3214,175.806 87.547,175.806 87.7727,175.806 87.9983,175.806 88.2239,175.805 
 88.4495,175.805 88.6752,175.805 88.9008,175.805 89.1264,175.804 89.352,175.804 89.5777,175.804 89.8033,175.804 90.0289,175.803 90.2545,175.803 90.4802,175.803 
 90.7058,175.

In [12]:
demo(GenerateDatasets.Likas2(), [64,64], 20_000)

loglikelihood(dataset, data) = -Inf

2017-09-13 12:42:53.742611: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)



(ii, ysmin, ysmax, condition_loss) = (1, [1.0034], [1.0034], [0.993221])
(ii, ysmin, ysmax, condition_loss) = (51, [1.17272], [1.17288], [0.713956])
(ii, ysmin, ysmax, condition_loss) = (101, [1.51344], [1.52236], [0.491755])
(ii, ysmin, ysmax, condition_loss) = (151, [1.50645], [1.53459], [0.473095])
(ii, ysmin, ysmax, condition_loss) = (201, [1.50309], [1.57218], [0.436127])
(ii, ysmin, ysmax, condition_loss) = (251, [1.49429], [1.61216], [0.394743])
(ii, ysmin, ysmax, condition_loss) = (301, [1.48483], [1.65756], [0.352327])
(ii, ysmin, ysmax, condition_loss) = (351, [1.47051], [1.70443], [0.30874])
(ii, ysmin, ysmax, condition_loss) = (401, [1.45049], [1.75069], [0.265095])
(ii, ysmin, ysmax, condition_loss) = (451, [1.42552], [1.79398], [0.223511])
(ii, ysmin, ysmax, condition_loss) = (501, [1.39824], [1.83094], [0.187177])
(ii, ysmin, ysmax, condition_loss) = (551, [1.37155], [1.86034], [0.157559])
(ii, ysmin, ysmax, condition_loss) = (601, [1.34708], [1.88303], [0.134145])
(ii,

Epoch 6701: loss: Any[[3.34005], [6.24595], [-7252.92], [7253.28], nothing]
Epoch 6801: loss: Any[[3.59366], [6.28289], [-7249.1], [7249.38], nothing]
Epoch 6901: loss: Any[[3.90447], [6.34627], [-7245.33], [7245.54], nothing]
Epoch 7001: loss: Any[[4.29282], [6.44746], [-7241.6], [7241.73], nothing]
Epoch 7101: loss: Any[[4.78633], [6.60822], [-7237.88], [7237.94], nothing]
Epoch 7201: loss: Any[[5.41861], [6.86293], [-7234.11], [7234.13], nothing]
Epoch 7301: loss: Any[[6.22441], [7.25597], [-7230.19], [7230.19], nothing]
Epoch 7401: loss: Any[[7.22701], [7.84442], [-7225.92], [7225.94], nothing]
Epoch 7501: loss: Any[[8.42404], [8.71725], [-7221.03], [7221.08], nothing]
Epoch 7601: loss: Any[[9.80793], [9.93087], [-7215.34], [7215.42], nothing]
Epoch 7701: loss: Any[[11.3992], [11.4367], [-7208.33], [7208.42], nothing]
Epoch 7801: loss: Any[[13.2564], [13.2652], [-7200.08], [7200.18], nothing]
Epoch 7901: loss: Any[[15.3908], [15.3932], [-7194.12], [7194.22], nothing]
Epoch 8001: lo

Epoch 17501: loss: Any[[184.883], [184.885], [-7111.86], [7111.96], nothing]
Epoch 17601: loss: Any[[195.567], [195.578], [-7117.66], [7117.75], nothing]
Epoch 17701: loss: Any[[198.288], [198.296], [-7116.19], [7116.29], nothing]
Epoch 17801: loss: Any[[208.852], [208.858], [-7114.33], [7114.43], nothing]
Epoch 17901: loss: Any[[225.39], [225.393], [-7111.37], [7111.47], nothing]
Epoch 18001: loss: Any[[240.897], [240.923], [-7120.15], [7120.24], nothing]
Epoch 18101: loss: Any[[243.798], [243.822], [-7118.34], [7118.44], nothing]
Epoch 18201: loss: Any[[246.851], [246.872], [-7117.52], [7117.62], nothing]
Epoch 18301: loss: Any[[250.074], [250.091], [-7116.58], [7116.67], nothing]
Epoch 18401: loss: Any[[253.489], [253.503], [-7115.43], [7115.53], nothing]
Epoch 18501: loss: Any[[257.145], [257.156], [-7114.0], [7114.1], nothing]
Epoch 18601: loss: Any[[258.072], [258.079], [-7112.01], [7112.1], nothing]
Epoch 18701: loss: Any[[296.167], [296.171], [-7108.56], [7108.66], nothing]
Epo

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,175.813 32.8746,175.813 33.5271,175.813 34.1796,175.813 34.8321,175.813 35.4846,175.813 36.1371,175.813 36.7895,175.813 37.442,175.813 38.0945,175.813 
 38.747,175.813 39.3995,175.813 40.052,175.813 40.7045,175.813 41.357,175.813 42.0095,175.813 42.662,175.813 43.3145,175.813 43.967,175.813 44.6194,175.813 
 45.2719,175.813 45.9244,175.813 46.5769,175.813 47.2294,175.813 47.8819,175.813 48.5344,175.813 49.1869,175.813 49.8394,175.813 50.4919,175.813 51.1444,175.813 
 51.7969,175.813 52.4493,175.813 53.1018,175.813 53.7543,175.813 54.4068,175.813 55.0593,175.813 55.7118,175.813 56.3643,175.813 57.0168,175.813 57.6693,175.813 
 58.3218,175.813 58.9743,175.813 59.6267,175.813 60.2792,175.813 60.9317,175.813 61.5842,175.813 62.2367,175.813 62.8892,175.813 63.5417,175.813 64.1942,175.813 
 64.8467,175.813 65.4992,175.813 66.1517,175.813 66.8042,175.812 67.4566,175.812 68.1091,175.812 68.7616,175.812 69.4141,175.812 70.0666,175.812 70.7191,175.812 
 71.3716,175.812 72.0241,175.812 72.6766,175.812 73.3291,175.812 73.9816,175.812 74.6341,175.812 75.2865,175.812 75.939,175.812 76.5915,175.812 77.244,175.812 
 77.8965,175.812 78.549,175.812 79.2015,175.812 79.854,175.811 80.5065,175.811 81.159,175.811 81.8115,175.811 82.464,175.811 83.1164,175.811 83.7689,175.811 
 84.4214,175.811 85.0739,175.811 85.7264,175.811 86.3789,175.811 87.0314,175.811 87.6839,175.811 88.3364,175.81 88.9889,175.81 89.6414,175.81 90.2938,175.81 
 90.9463,175.81 91.5988,175.81 92.2513,175.81 92.9038,175.81 93.5563,175.809 94.2088,175.809 94.8613,175.809 95.5138,175.809 96.1663,175.809 96.8188,175.809 
 97.4713,175.808 98.1237,175.808 98.7762,175.808 99.4287,175.808 100.081,175.808 100.734,175.808 101.386,175.807 102.039,175.807 102.691,175.807 103.344,175.807 
 103.996,175.806 104.649,175.806 105.301,175.806 105.954,175.805 106.606,175.805 107.259,175.805 107.911,175.804 108.564,175.804 109.216,175.804 109.869,175.803 
 110.521,175.803 111.174,175.802 111.826,175.802 112.479,175.801 113.131,175.801 113.784,175.8 114.436,175.8 115.089,175.799 115.741,175.799 116.394,175.798 
 117.046,175.797 117.698,175.796 118.351,175.796 119.003,175.795 119.656,175.794 120.308,175.793 120.961,175.792 121.613,175.79 122.266,175.789 122.918,175.788 
 123.571,175.786 124.223,175.785 124.876,175.783 125.528,175.781 126.181,175.779 126.833,175.777 127.486,175.775 128.138,175.772 128.791,175.769 129.443,175.766 
 130.096,175.762 130.748,175.758 131.401,175.754 132.053,175.749 132.706,175.743 133.358,175.736 134.011,175.728 134.663,175.719 135.316,175.708 135.968,175.695 
 136.621,175.679 137.273,175.659 137.926,175.635 138.578,175.603 139.231,175.561 139.883,175.503 140.536,175.416 141.188,175.27 141.841,174.905 142.493,172.398 
 143.146,137.964 143.798,11.811 144.451,101.754 145.103,92.3933 145.756,67.5091 146.408,61.8066 147.061,71.6761 147.713,81.6323 148.366,85.299 149.018,84.0421 
 149.671,80.839 150.323,77.8547 150.976,76.1258 151.628,75.8188 152.281,76.5912 152.933,77.9183 153.586,79.3179 154.238,80.4641 154.891,81.2062 155.543,81.5372 
 156.195,81.5435 156.848,81.3588 157.5,81.1247 158.153,80.9678 158.805,80.9844 159.458,81.2355 160.11,81.7458 160.763,82.5098 161.415,83.4969 162.068,84.6593 
 162.72,85.9399 163.373,87.2785 164.025,88.6174 164.678,89.9049 165.33,91.0982 165.983,92.1643 166.635,93.0801 167.288,93.8319 167.94,94.415 168.593,94.8313 
 169.245,95.0892 169.898,95.2013 170.55,95.1839 171.203,95.0552 171.855,94.8348 172.508,94.5431 173.16,94.2 173.813,93.8248 174.465,93.4358 175.118,93.05 
 175.77,92.6824 176.423,92.3465 177.075,92.0539 177.728,91.8138 178.38,91.6342 179.033,91.5204 179.685,91.4765 180.338,91.5043 180.99,91.6046 181.643,91.7763 
 182.295,92.0169 

In [13]:
demo(GenerateDatasets.MagdonIsmailAndAtiya(), [32], 20_000)

loglikelihood(dataset, data) = -770.7510992184657


2017-09-13 12:44:30.148538: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)


(ii, ysmin, ysmax, condition_loss) = (1, [1.00139], [1.0014], [0.997201])
(ii, ysmin, ysmax, condition_loss) = (51, [1.06685], [1.08602], [0.839835])
(ii, ysmin, ysmax, condition_loss) = (101, [1.1688], [1.69041], [0.124343])
(ii, ysmin, ysmax, condition_loss) = (151, [1.10701], [1.97889], [0.0118964])
(ii, ysmin, ysmax, condition_loss) = (201, [1.07624], [1.99761], [0.00581817])
(ii, ysmin, ysmax, condition_loss) = (251, [1.05997], [1.99886], [0.00359727])
(ii, ysmin, ysmax, condition_loss) = (301, [1.04973], [1.99918], [0.00247372])
(ii, ysmin, ysmax, condition_loss) = (351, [1.04263], [1.99935], [0.0018175])
(ii, ysmin, ysmax, condition_loss) = (401, [1.03738], [1.99947], [0.00139762])
(ii, ysmin, ysmax, condition_loss) = (451, [1.03333], [1.99955], [0.00111123])
(ii, ysmin, ysmax, condition_loss) = (501, [1.0301], [1.99962], [0.000906399])
(ii, ysmin, ysmax, condition_loss) = (551, [1.02746], [1.99967], [0.000754416])
(ii, ysmin, ysmax, condition_loss) = (601, [1.02526], [1.99971],

Epoch 6701: loss: Any[[1.0], [1.0013], [-791.011], [791.111], nothing]
Epoch 6801: loss: Any[[1.0], [1.00131], [-790.527], [790.627], nothing]
Epoch 6901: loss: Any[[1.0], [1.00128], [-790.052], [790.152], nothing]
Epoch 7001: loss: Any[[1.0], [1.00131], [-789.586], [789.686], nothing]
Epoch 7101: loss: Any[[1.0], [1.00129], [-789.128], [789.228], nothing]
Epoch 7201: loss: Any[[1.0], [1.00126], [-788.679], [788.778], nothing]
Epoch 7301: loss: Any[[1.0], [1.00126], [-788.241], [788.341], nothing]
Epoch 7401: loss: Any[[1.0], [1.00126], [-787.805], [787.905], nothing]
Epoch 7501: loss: Any[[1.0], [1.00124], [-787.381], [787.48], nothing]
Epoch 7601: loss: Any[[1.0], [1.00125], [-786.964], [787.064], nothing]
Epoch 7701: loss: Any[[1.0], [1.00123], [-786.555], [786.655], nothing]
Epoch 7801: loss: Any[[1.0], [1.00121], [-786.155], [786.254], nothing]
Epoch 7901: loss: Any[[1.0], [1.00123], [-785.761], [785.861], nothing]
Epoch 8001: loss: Any[[1.0], [1.00121], [-785.376], [785.475], not

Epoch 18101: loss: Any[[1.0], [1.00207], [-770.917], [771.017], nothing]
Epoch 18201: loss: Any[[1.0], [1.00211], [-770.911], [771.01], nothing]
Epoch 18301: loss: Any[[1.0], [1.00213], [-770.904], [771.004], nothing]
Epoch 18401: loss: Any[[1.0], [1.00216], [-770.899], [770.999], nothing]
Epoch 18501: loss: Any[[1.0], [1.00218], [-770.894], [770.994], nothing]
Epoch 18601: loss: Any[[1.0], [1.00221], [-770.89], [770.99], nothing]
Epoch 18701: loss: Any[[1.0], [1.00223], [-770.887], [770.986], nothing]
Epoch 18801: loss: Any[[1.0], [1.00226], [-770.884], [770.983], nothing]
Epoch 18901: loss: Any[[1.0], [1.00229], [-770.881], [770.981], nothing]
Epoch 19001: loss: Any[[1.0], [1.00232], [-770.879], [770.979], nothing]
Epoch 19101: loss: Any[[1.0], [1.00235], [-770.877], [770.977], nothing]
Epoch 19201: loss: Any[[1.0], [1.00237], [-770.877], [770.977], nothing]
Epoch 19301: loss: Any[[1.0], [1.00241], [-770.875], [770.974], nothing]
Epoch 19401: loss: Any[[1.0], [1.00243], [-770.874], [

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -50 
 
 
 -25 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.8117,175.813 38.8659,175.813 38.92,175.813 38.9742,175.813 39.0283,175.813 39.0825,175.813 39.1366,175.813 39.1908,175.813 39.2449,175.813 39.2991,175.813 
 39.3532,175.813 39.4074,175.813 39.4615,175.813 39.5157,175.812 39.5698,175.812 39.624,175.812 39.6781,175.812 39.7323,175.812 39.7864,175.812 39.8406,175.812 
 39.8947,175.812 39.9489,175.812 40.003,175.812 40.0572,175.812 40.1113,175.812 40.1655,175.811 40.2196,175.811 40.2738,175.811 40.3279,175.811 40.3821,175.811 
 40.4362,175.811 40.4904,175.811 40.5445,175.811 40.5987,175.811 40.6528,175.811 40.707,175.811 40.7611,175.811 40.8153,175.81 40.8694,175.81 40.9236,175.81 
 40.9777,175.81 41.0319,175.81 41.086,175.81 41.1402,175.81 41.1943,175.81 41.2485,175.81 41.3027,175.81 41.3568,175.81 41.411,175.809 41.4651,175.809 
 41.5193,175.809 41.5734,175.809 41.6276,175.809 41.6817,175.809 41.7359,175.809 41.79,175.809 41.8442,175.809 41.8983,175.809 41.9525,175.808 42.0066,175.808 
 42.0608,175.808 42.1149,175.808 42.1691,175.808 42.2232,175.808 42.2774,175.808 42.3315,175.808 42.3857,175.808 42.4398,175.808 42.494,175.807 42.5481,175.807 
 42.6023,175.807 42.6564,175.807 42.7106,175.807 42.7647,175.807 42.8189,175.807 42.873,175.807 42.9272,175.807 42.9813,175.807 43.0355,175.806 43.0896,175.806 
 43.1438,175.806 43.1979,175.806 43.2521,175.806 43.3062,175.806 43.3604,175.806 43.4145,175.806 43.4687,175.806 43.5228,175.805 43.577,175.805 43.6311,175.805 
 43.6853,175.805 43.7394,175.805 43.7936,175.805 43.8477,175.805 43.9019,175.805 43.956,175.804 44.0102,175.804 44.0643,175.804 44.1185,175.804 44.1726,175.804 
 44.2268,175.804 44.2809,175.804 44.3351,175.804 44.3892,175.803 44.4434,175.803 44.4975,175.803 44.5517,175.803 44.6058,175.803 44.66,175.803 44.7141,175.803 
 44.7683,175.803 44.8224,175.802 44.8766,175.802 44.9307,175.802 44.9849,175.802 45.039,175.802 45.0932,175.802 45.1473,175.802 45.2015,175.802 45.2556,175.801 
 45.3098,175.801 45.3639,175.801 45.4181,175.801 45.4722,175.801 45.5264,175.801 45.5805,175.801 45.6347,175.8 45.6888,175.8 45.743,175.8 45.7971,175.8 
 45.8513,175.8 45.9054,175.8 45.9596,175.8 46.0137,175.799 46.0679,175.799 46.122,175.799 46.1762,175.799 46.2303,175.799 46.2845,175.799 46.3386,175.799 
 46.3928,175.798 46.4469,175.798 46.5011,175.798 46.5552,175.798 46.6094,175.798 46.6635,175.798 46.7177,175.797 46.7718,175.797 46.826,175.797 46.8801,175.797 
 46.9343,175.797 46.9884,175.797 47.0426,175.797 47.0967,175.796 47.1509,175.796 47.205,175.796 47.2592,175.796 47.3133,175.796 47.3675,175.796 47.4216,175.795 
 47.4758,175.795 47.5299,175.795 47.5841,175.795 47.6382,175.795 47.6924,175.795 47.7465,175.794 47.8007,175.794 47.8548,175.794 47.909,175.794 47.9631,175.794 
 48.0173,175.793 48.0714,175.793 48.1256,175.793 48.1797,175.793 48.2339,175.793 48.288,175.793 48.3422,175.792 48.3963,175.792 48.4505,175.792 48.5046,175.792 
 48.5588,175.792 48.6129,175.791 48.6671,175.791 48.7212,175.791 48.7754,175.791 48.8295,175.791 48.8837,175.791 48.9378,175.79 48.992,175.79 49.0461,175.79 
 49.1003,175.79 49.1544,175.79 49.2086,175.789 49.2627,175.789 49.3169,175.789 49.371,175.789 49.4252,175.789 49.4793,175.788 49.5335,175.788 49.5876,175.788 
 49.6418,175.788 49.696,175.788 49.7501,175.787 49.8043,175.787 49.8584,175.787 49.9126,175.787 49.9667,175.787 50.0209,175.786 50.075,175.786 50.1292,175.786 
 50.1833,175.786 50.2375,175.785 50.2916,175.785 50.3458,175.785 50.3999,175.785 50.4541,175.785 50.5082,175.784 50.5624,175.784 50.6165,175.784 50.6707,175.784 
 50.7248,175.783 50.779,175.783 50.8331,175.783 50.8873,175.783 50.9414,175.782 50.9956,175.782 51.0497,175.782 51.1039,175.782 51.158,175.782 51.2122,175.781 
 51.2663,175.78

In [14]:
demo(Arcsine(1,4), [64,64], 20_000)

loglikelihood(dataset, data) = -4221.273645560377


2017-09-13 12:45:30.740486: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)


(ii, ysmin, ysmax, condition_loss) = (1, [1.00285], [1.00285], [0.994308])
(ii, ysmin, ysmax, condition_loss) = (51, [1.16884], [1.1689], [0.719235])
(ii, ysmin, ysmax, condition_loss) = (101, [1.51288], [1.51629], [0.497026])
(ii, ysmin, ysmax, condition_loss) = (151, [1.50377], [1.51318], [0.49078])
(ii, ysmin, ysmax, condition_loss) = (201, [1.50515], [1.52687], [0.479023])
(ii, ysmin, ysmax, condition_loss) = (251, [1.50384], [1.54467], [0.461181])
(ii, ysmin, ysmax, condition_loss) = (301, [1.50067], [1.56102], [0.443382])
(ii, ysmin, ysmax, condition_loss) = (351, [1.49876], [1.57851], [0.426413])
(ii, ysmin, ysmax, condition_loss) = (401, [1.49694], [1.59707], [0.4093])
(ii, ysmin, ysmax, condition_loss) = (451, [1.49435], [1.61589], [0.391922])
(ii, ysmin, ysmax, condition_loss) = (501, [1.49087], [1.63526], [0.373992])
(ii, ysmin, ysmax, condition_loss) = (551, [1.48629], [1.65565], [0.355057])
(ii, ysmin, ysmax, condition_loss) = (601, [1.48029], [1.67756], [0.334648])
(ii, y

Epoch 6901: loss: Any[[1.00234], [7.28106], [-5115.34], [5118.13], nothing]
Epoch 7001: loss: Any[[1.00227], [7.21491], [-5105.77], [5108.48], nothing]
Epoch 7101: loss: Any[[1.00219], [7.15157], [-5096.36], [5099.01], nothing]
Epoch 7201: loss: Any[[1.00211], [7.09083], [-5087.1], [5089.69], nothing]
Epoch 7301: loss: Any[[1.00203], [7.03199], [-5077.98], [5080.51], nothing]
Epoch 7401: loss: Any[[1.00195], [6.97413], [-5068.99], [5071.46], nothing]
Epoch 7501: loss: Any[[1.00186], [6.9179], [-5060.11], [5062.53], nothing]
Epoch 7601: loss: Any[[1.00177], [6.8633], [-5051.33], [5053.7], nothing]
Epoch 7701: loss: Any[[1.00169], [6.81002], [-5042.66], [5044.98], nothing]
Epoch 7801: loss: Any[[1.00161], [6.7591], [-5034.09], [5036.35], nothing]
Epoch 7901: loss: Any[[1.00154], [6.71178], [-5025.6], [5027.82], nothing]
Epoch 8001: loss: Any[[1.00148], [6.66652], [-5017.2], [5019.37], nothing]
Epoch 8101: loss: Any[[1.00143], [6.62373], [-5008.86], [5011.0], nothing]
Epoch 8201: loss: An

Epoch 17701: loss: Any[[1.00096], [4.11292], [-4654.81], [4655.26], nothing]
Epoch 17801: loss: Any[[1.00095], [4.09374], [-4652.58], [4653.02], nothing]
Epoch 17901: loss: Any[[1.00093], [4.07431], [-4650.38], [4650.81], nothing]
Epoch 18001: loss: Any[[1.00091], [4.05481], [-4648.2], [4648.62], nothing]
Epoch 18101: loss: Any[[1.00089], [4.03538], [-4646.05], [4646.46], nothing]
Epoch 18201: loss: Any[[1.00088], [4.0154], [-4643.92], [4644.33], nothing]
Epoch 18301: loss: Any[[1.00086], [3.99659], [-4641.82], [4642.22], nothing]
Epoch 18401: loss: Any[[1.00084], [3.97794], [-4639.74], [4640.13], nothing]
Epoch 18501: loss: Any[[1.00082], [3.96005], [-4637.7], [4638.08], nothing]
Epoch 18601: loss: Any[[1.00081], [3.94176], [-4635.67], [4636.04], nothing]
Epoch 18701: loss: Any[[1.00079], [3.92384], [-4633.67], [4634.04], nothing]
Epoch 18801: loss: Any[[1.00078], [3.90561], [-4631.67], [4632.04], nothing]
Epoch 18901: loss: Any[[1.00076], [3.88887], [-4629.7], [4630.05], nothing]
Epo

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.5169,175.811 37.0302,175.81 38.5436,175.81 40.0569,175.809 41.5702,175.808 43.0836,175.807 44.5969,175.806 46.1102,175.805 47.6235,175.803 49.1369,175.801 
 50.6502,175.799 52.1635,175.796 53.6769,175.793 55.1902,175.789 56.7035,175.784 58.2168,175.778 59.7302,175.77 61.2435,175.76 62.7568,175.748 64.2702,175.732 
 65.7835,175.711 67.2968,175.684 68.8101,175.648 70.3235,175.597 71.8368,175.522 73.3501,175.407 74.8634,175.2 76.3768,174.688 77.8901,172.401 79.4034,153.924 
 80.9168,11.811 82.4301,13.6915 83.9434,95.3332 85.4567,79.3396 86.9701,112.127 88.4834,127.726 89.9967,122.617 91.5101,119.003 93.0234,124.477 94.5367,134 
 96.05,141.367 97.5634,144.877 99.0767,145.484 100.59,144.54 102.103,143.045 103.617,141.602 105.13,140.534 106.643,139.97 108.157,139.921 109.67,140.325 
 111.183,141.082 112.697,142.083 114.21,143.225 115.723,144.421 117.237,145.601 118.75,146.716 120.263,147.736 121.777,148.643 123.29,149.432 124.803,150.105 
 126.317,150.671 127.83,151.142 129.343,151.532 130.857,151.854 132.37,152.124 133.883,152.354 135.397,152.557 136.91,152.744 138.423,152.924 139.937,153.104 
 141.45,153.29 142.963,153.486 144.477,153.696 145.99,153.921 147.503,154.161 149.017,154.416 150.53,154.685 152.043,154.966 153.556,155.256 155.07,155.554 
 156.583,155.855 158.096,156.157 159.61,156.456 161.123,156.751 162.636,157.037 164.15,157.313 165.663,157.576 167.176,157.824 168.69,158.056 170.203,158.27 
 171.716,158.464 173.23,158.639 174.743,158.793 176.256,158.926 177.77,159.039 179.283,159.131 180.796,159.203 182.31,159.256 183.823,159.291 185.336,159.308 
 186.85,159.309 188.363,159.294 189.876,159.267 191.39,159.226 192.903,159.175 194.416,159.115 195.93,159.047 197.443,158.973 198.956,158.894 200.47,158.812 
 201.983,158.729 203.496,158.645 205.01,158.563 206.523,158.483 208.036,158.406 209.55,158.335 211.063,158.269 212.576,158.211 214.09,158.16 215.603,158.117 
 217.116,158.084 218.63,158.06 220.143,158.045 221.656,158.041 223.17,158.047 224.683,158.064 226.196,158.09 227.71,158.126 229.223,158.172 230.736,158.227 
 232.25,158.291 233.763,158.363 235.276,158.443 236.79,158.53 238.303,158.622 239.816,158.721 241.33,158.823 242.843,158.93 244.356,159.04 245.869,159.152 
 247.383,159.266 248.896,159.38 250.409,159.494 251.923,159.607 253.436,159.719 254.949,159.828 256.463,159.935 257.976,160.038 259.489,160.137 261.003,160.231 
 262.516,160.321 264.029,160.405 265.543,160.483 267.056,160.555 268.569,160.621 270.083,160.68 271.596,160.732 273.109,160.778 274.623,160.817 276.136,160.848 
 277.649,160.873 279.163,160.89 280.676,160.901 282.189,160.905 283.703,160.902 285.216,160.893 286.729,160.877 288.243,160.856 289.756,160.828 291.269,160.795 
 292.783,160.756 294.296,160.713 295.809,160.665 297.323,160.612 298.836,160.556 300.349,160.495 301.863,160.432 303.376,160.365 304.889,160.296 306.403,160.225 
 307.916,160.152 309.429,160.078 310.943,160.002 312.456,159.926 313.969,159.849 315.483,159.773 316.996,159.697 318.509,159.622 320.023,159.548 321.536,159.475 
 323.049,159.404 324.563,159.336 326.076,159.27 327.589,159.206 329.103,159.146 330.616,159.088 332.129,159.034 333.643,158.984 335.156,158.938 336.669,158.895 
 338.182,158.857 339.696,158.824 341.209,158.794 342.722,158.769 344.236,158.749 345.749,158.733 347.262,158.722 348.776,158.715 350.289,158.713 351.802,158.716 
 353.316,158.723 354.829,158.734 356.342,158.75 357.856,158.769 359.369,158.793 360.882,158.82 362.396,158.851 363.909,158.885 365.422,158.922 366.936,158.961 
 368.449,159.004 369.962,159.048 371.476,159.095 372.989,159.143 374.502,159.192 376.016,159.242 377.529,159.293 379.042,159.344 380.556,159.395 382.069,159.445 
 383.582,159.494 385.096,159.543 386.609

In [17]:
dataset = GenerateDatasets.Likas1()
data = original_sample(dataset)
@show loglikelihood(dataset, data)
est = NeuralDensityEstimator([64], approximate_support(dataset))
condition!(est)
println("Conditioning Done")
fit!(est, data; epochs=10_000)
println("Fitting Done")

loglikelihood(dataset, data) = -10537.403257872073


2017-09-13 17:28:33.470989: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)


(ii, ysmin, ysmax, condition_loss) = (1, [1.00338], [1.00339], [0.993242])
(ii, ysmin, ysmax, condition_loss) = (51, [1.1545], [1.16408], [0.722624])
(ii, ysmin, ysmax, condition_loss) = (101, [1.49345], [1.71246], [0.32617])
(ii, ysmin, ysmax, condition_loss) = (151, [1.36911], [1.86913], [0.15337])
(ii, ysmin, ysmax, condition_loss) = (201, [1.2604], [1.94453], [0.0708851])
(ii, ysmin, ysmax, condition_loss) = (251, [1.19565], [1.9708], [0.0391334])
(ii, ysmin, ysmax, condition_loss) = (301, [1.15656], [1.98198], [0.0248365])
(ii, ysmin, ysmax, condition_loss) = (351, [1.13074], [1.98768], [0.0172436])
(ii, ysmin, ysmax, condition_loss) = (401, [1.11241], [1.991], [0.0127168])
(ii, ysmin, ysmax, condition_loss) = (451, [1.0987], [1.99312], [0.0097899])
(ii, ysmin, ysmax, condition_loss) = (501, [1.08805], [1.99455], [0.00778196])
(ii, ysmin, ysmax, condition_loss) = (551, [1.07951], [1.99557], [0.00634098])
(ii, ysmin, ysmax, condition_loss) = (601, [1.0725], [1.99633], [0.00526967])

Epoch 6701: loss: Any[[1.00001], [1.0025], [-14744.1], [14744.2], nothing]
Epoch 6801: loss: Any[[1.00001], [1.0025], [-14733.1], [14733.2], nothing]
Epoch 6901: loss: Any[[1.00001], [1.00245], [-14722.5], [14722.6], nothing]
Epoch 7001: loss: Any[[1.00001], [1.00241], [-14712.1], [14712.2], nothing]
Epoch 7101: loss: Any[[1.00001], [1.00244], [-14701.4], [14701.5], nothing]
Epoch 7201: loss: Any[[1.00001], [1.0024], [-14691.1], [14691.2], nothing]
Epoch 7301: loss: Any[[1.00001], [1.00236], [-14680.8], [14680.9], nothing]
Epoch 7401: loss: Any[[1.00001], [1.00235], [-14670.7], [14670.8], nothing]
Epoch 7501: loss: Any[[1.00001], [1.00232], [-14660.7], [14660.8], nothing]
Epoch 7601: loss: Any[[1.00001], [1.00229], [-14650.8], [14650.9], nothing]
Epoch 7701: loss: Any[[1.00001], [1.0023], [-14640.9], [14641.0], nothing]
Epoch 7801: loss: Any[[1.00001], [1.00227], [-14631.2], [14631.3], nothing]
Epoch 7901: loss: Any[[1.00001], [1.00227], [-14621.6], [14621.7], nothing]
Epoch 8001: loss

In [31]:
gr =est.sess.graph
run(est.sess, exp(6*gr["W_2"].^2 + gr["b_2"]))

64×1 Array{Float32,2}:
 4.26033  
 3.74189  
 4.08921  
 4.00873  
 3.83911  
 3.98454  
 4.03264  
 4.24852  
 1.22124f8
 3.94933  
 4.22194  
 4.08248  
 4.2943   
 ⋮        
 3.45641  
 4.0339   
 3.98625  
 4.08046  
 3.97232  
 3.6441   
 4.13142  
 4.01877  
 3.92249  
 4.30205  
 4.08415  
 3.98094  

In [19]:
collect(keys(est.sess.graph))

826-element Array{String,1}:
 "t"                 
 "smin"              
 "smax"              
 "W_2"               
 "W_2/Assign/Const"  
 "W_2/Assign"        
 "W_2_squared"       
 "b_2"               
 "b_2/Assign/Const_2"
 "b_2/Assign"        
 "W_3"               
 "W_3/Assign/Const_3"
 "W_3/Assign"        
 ⋮                   
 "Mul_38"            
 "Add_26"            
 "Sqrt_12"           
 "Const_105"         
 "Add_27"            
 "Div_12"            
 "Mul_39"            
 "AssignSub_6"       
 "Assign_17"         
 "Assign_18"         
 "Group_2"           
 "Group_3"           